# Mayak 1958 criticality accident

In 1958, following a series of criticality experiments, a large tank (75cm in diameter, 100cm in height), containing uranyl nitrate solution was emptied. The experimenters first, following the safety procedures, started draining the contents of the tank into small, favorable geometry bottles with a volume of 6 liters each. After several bottles were filled, the experimenters considered the remaining solution in the large tank to be safely subcritical, therefore decided to speed up the process by unbolting the tank from its stand, in order to directly pour the solution into the bottles. Although the exact movement of the tank is unknown, it is certain that during the movement the solution in the tank became prompt supercritical, and the solution was ejected from the tank and hit the ceiling 5 meters above. The experimenters received lethal dose of radiation, except one, who had acute radiation thickness.

The accident is described in more detail in the report [LA-13638](https://www.osti.gov/biblio/758324).

Your task is to investigate with OpenMC how the movement of the container might have impacted the effective multiplication factor of the system. 
1. study the impact of the tilting angle (this will influence the surface to volume ratio). Calculate the multiplication factor for every 2degrees up to the point the liquid reaches the top of the cylinder
2. study the impact of lifting the container from the stand (this will influence the reflection). The stand can be assumed to be concrete.
3. Alternatively to 2., you can consider the container to be closed, and evaluate the multiplication factor for each angles between standing and lying cylinder (see note below on the height of the liquid)

You can use the following assumptions:

1. inner diameter and height of the container: 75cm, 100cm
2. The UN solution consists of the following isotopes and weight percentage (based on [Vega et al.](https://www.osti.gov/biblio/1255788))
   -  'H1', 6.279
   -  'H2', 0.001
   - 'N14', 3.284
   - 'N15', 0.013
   - 'O16', 64.387
   - 'O18', 0.149
   - 'U235', 23.287
   - 'U238', 2.587
   - density (g/cm3): 1.5477)
3. the height of the liquid in standing position was 13.2cm
4. the movements are ideal (the shape of the liquid has it's steady state shape, no sloshing to be considered)

The height of the liquid at a tilted position is not trivial, therefore you find the `getplane.py` module along this notebook. The module is able to estimate the S parameter (see figure below, note, the parameter is negative for partially covered bottom) based on the tilting angle $\Phi$, and gives the equation of the surface (as Ax+By+Cz-D=0)). Consider that the module only estimates the height for cases when the bottom is fully covered and the top is dry (such as in the figure), or when the bottom is partially covered and the top is dry. It is considered here that the top is open, therefore the liquid would pour out from the container if the surface reaches the top. Therefore in your tilting angle study you should not investigate angles for which this would happen. However, you are welcome to extend the module to closed containers and handle cases when the bottom is fully cover and the top is partially covered, and when both the top and bottom are only partially covered. If you choose to do that you can skip the reflection study, and only focus on the tilting angle study.

Usage of the module: Calculate S for the target liquid volume, the container parameters and the tipping angle. Then evaluate the plane. In order to know what sign to be used in openMC, evaluate the sign for a coordinate which you know is in the region of your interest. Note that the module considers the coordinate system always attached to the container (ie. z-axis points from bottom to top), so the container itself is not actually tipped, only the liquid surface is rotated. You can consider the same in openMC. However, when you consider the reflection from the concrete stand, then the rotation of that needs to be taken into account.

```python
    from getplane import *
    S = getS(Vl, r, p, Hc) #liquid volume, radius of container, angle (deg), container height
    A,B,C,D = getPlane(r,p,S) #radius of container, angle (deg), S parameter
    pSign=getPlaneSign((x0,y0,z0),A,B,C,D) #test coordinate, plane coefficients
```

Note, that when modelling the geometry, having the right initial source might require some tinkering to make sure that you never end up with an initial source outside of your system, or loosing too much source particles. You can try to sample locations in the volume (eg. sample in the whole container or a bounding cylinder and reject locations above the liquid surface), or always place the source in the center of the liquid.

![](geometry.png)

